In [0]:
from pyspark.sql import SparkSession, functions as F
spark = SparkSession.builder.appName("Option_2_level_2").getOrCreate()

In [0]:
temp = spark.read.table("delta_silver_valid")
delta_silver_valid_temp = temp.drop("is_valid_record")


if not spark.catalog.tableExists("delta_facts_sales"):
    
    print("Table not exists")

    delta_silver_valid_temp \
    .withColumn("ingestion_timestamp", F.current_timestamp()) \
    .write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("delta_facts_sales")
    
    print("IF:Number of records in the table: ", delta_silver_valid_temp.count())


else:
    print("Table exists")

    delta_facts_sales_temp = spark.read.table("delta_facts_sales")
    delta_facts_sales_id = delta_facts_sales_temp.select("ShipmentID")

    delta_join = delta_silver_valid_temp.join(delta_facts_sales_id, on = "ShipmentID", how = "left_anti")
    
    delta_join \
    .withColumn("ingestion_timestamp", F.current_timestamp()) \
    .write.format("delta") \
    .mode("append") \
    .saveAsTable("delta_facts_sales")

    print("No. of rows to append: ", delta_join.count())
    
    temp_check = spark.read.table("delta_facts_sales")
    print("else:Number of records in the table: ", temp_check.count())

    temp_check.show(2)

In [0]:
%sql
show tables;